### Imports

In [33]:
import csv
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string
import uritools
import urlextract
from langdetect import detect
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import normalize

### Reading Data

In [34]:
extractor = urlextract.URLExtract()
ps = PorterStemmer()

data = {'airline_sentiment':[],'text':[]}
airline_sentiment = []
corpus = []

with open('Tweets.csv', 'r', encoding='utf8') as f:
    tweets = csv.reader(f)
    for row in tweets:        
        data['airline_sentiment'].append(row[1])
        data['text'].append(row[10])

print("Data Read")

Data Read


### Normalizing, Stemming, CaseFolding and Removing Stop Words from Text

In [35]:
def clean(words):
    urls = extractor.find_urls(words+" ")
    for url in urls:
        words = words.replace(url,'')
    tknzr = TweetTokenizer()
    words = tknzr.tokenize(words)
    exclude = set(string.punctuation)
    words = [word.lower() for word in words if not word.lower() in exclude]
    words = [word.lower() for word in words 
            if not word in set(stopwords.words('english')) and not word.isdigit()]
    words = [ps.stem(word) for word in words]
    words = ' '.join(words)
    return words

### Filtering Text

In [36]:
def CleanWithoutFilter():
    corpus = []
    corpusText=''
    with open('Tweets.csv',  encoding='utf8') as File:
        spamreader = csv.reader(File)
        for row in spamreader:       
            corpusText =  clean(row[10])
            corpus.append(corpusText)
    return corpus

def CleanWithFilter():
    corpus = []
    corpusText=''
    counter = 0
    with open('Tweets.csv',  encoding='utf8') as File:
        spamreader = csv.reader(File)
        for row in spamreader:       
            corpusText =  clean(row[10])
            if(not(corpusText.__contains__("RT") or (len(corpusText )<20) or (detect(row[10])=="en"))):
                corpus.append(corpusText)
                counter+=1
            else:
                del data['airline_sentiment'][counter]
    return corpus

### Tf-idf Vectorizer

In [37]:
def vectorizerFunction(filterOrNoFilter):
    vectorizer = TfidfVectorizer(stop_words='english')
    X_train, X_test, y_train, y_test = train_test_split(filterOrNoFilter,  data['airline_sentiment'], test_size = 0.2)
    vectorizer.fit(X_train)
    XTrain = vectorizer.transform(X_train)
    XTest = vectorizer.transform(X_test)
    return XTrain, XTest, y_train, y_test

### Multinomial Naive Bayes Classifier

In [38]:
def MNBClassifier(XTrain, XTest, y_train, y_test):
    clf = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
    clf.fit(XTrain, y_train)
    predictions = clf.predict(XTest)
    score = f1_score(y_test, predictions, average = 'micro')  
    print(score)

### K Nearest Neighbour Classifier

In [39]:
def KNeighbourClassifiers(XTrain, XTest, y_train, y_test):
    neigh = KNeighborsClassifier(n_neighbors = 5)
    neigh.fit(XTrain, y_train) 
    predictions = neigh.predict(XTest)
    score = f1_score(y_test, predictions, average = 'micro')  
    print(score)

### Random Forest Classifier

In [40]:
def RForestClassifiers(XTrain, XTest, y_train, y_test):
    clf = RandomForestClassifier(random_state = 0)
    clf.fit(XTrain, y_train)
    predictions = clf.predict(XTest)
    score = f1_score(y_test, predictions, average = 'micro')  
    print(score)

### Calling Tfidf Vectorizer without Filtering

In [41]:
XTrain, XTest, y_train, y_test = vectorizerFunction(CleanWithoutFilter())

### Classifiers without Filter F1_Score

In [42]:
MNBClassifier(XTrain, XTest, y_train, y_test)

0.6705360191191533


In [43]:
KNeighbourClassifiers(XTrain, XTest, y_train, y_test)

0.702628883578013


In [44]:
RForestClassifiers(XTrain, XTest, y_train, y_test)

c:\users\andrew\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7412086036189826


### Calling Tfidf Vectorizer with Filtering

In [45]:
XTrain, XTest, y_train, y_test = vectorizerFunction(CleanWithFilter())

### Classifiers with Filter F1_Score

In [46]:
MNBClassifier(XTrain, XTest, y_train, y_test)

0.625


In [47]:
KNeighbourClassifiers(XTrain, XTest, y_train, y_test)

0.5833333333333334


In [48]:
RForestClassifiers(XTrain, XTest, y_train, y_test)

0.5833333333333334


c:\users\andrew\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
